Beat-n game with 10 pt limit

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using JLD2

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const MXV = 9
const MNV = 1

1

In [4]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [5]:


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [6]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 10 and filter nondominated

In [7]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [8]:
lib0 = get_library(10)
lib0

126-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 1, 5, 999)
 Spud("", 1, 1, 1, 1, 2, 4, 999)
 Spud("", 1, 1, 1, 1, 3, 3, 999)
 Spud("", 1, 1, 1, 1, 4, 2, 999)
 Spud("", 1, 1, 1, 1, 5, 1, 999)
 Spud("", 1, 1, 1, 2, 1, 4, 999)
 Spud("", 1, 1, 1, 2, 2, 3, 999)
 Spud("", 1, 1, 1, 2, 3, 2, 999)
 Spud("", 1, 1, 1, 2, 4, 1, 999)
 Spud("", 1, 1, 1, 3, 1, 3, 999)
 Spud("", 1, 1, 1, 3, 2, 2, 999)
 Spud("", 1, 1, 1, 3, 3, 1, 999)
 Spud("", 1, 1, 1, 4, 1, 2, 999)
 ⋮
 Spud("", 3, 1, 3, 1, 1, 1, 999)
 Spud("", 3, 2, 1, 1, 1, 2, 999)
 Spud("", 3, 2, 1, 1, 2, 1, 999)
 Spud("", 3, 2, 1, 2, 1, 1, 999)
 Spud("", 3, 2, 2, 1, 1, 1, 999)
 Spud("", 3, 3, 1, 1, 1, 1, 999)
 Spud("", 4, 1, 1, 1, 1, 2, 999)
 Spud("", 4, 1, 1, 1, 2, 1, 999)
 Spud("", 4, 1, 1, 2, 1, 1, 999)
 Spud("", 4, 1, 2, 1, 1, 1, 999)
 Spud("", 4, 2, 1, 1, 1, 1, 999)
 Spud("", 5, 1, 1, 1, 1, 1, 999)

In [9]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [10]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [11]:
lib=filter_nondominated2(lib0)
for ff in lib
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end
nlib = length(lib)

 111124 111133 111142 111214 111223 111232 111241 111313 111322 111331 111412 111421 111511 112114 112123 112132 112141 112213 112222 112231 112312 112321 112411 113113 113122 113131 113212 113221 113311 114112 114121 114211 121114 121123 121132 121213 121222 121231 121312 121321 121411 122113 122122 122212 122221 122311 123112 123211 131113 131122 131212 131221 131311 132112 132211 141112 141211 212113 212122 212131 212212 212221 212311 213112 213121 213211 214111 222112 222211 312112 312121 312211 313111 412111

74

In [12]:
payoffs = get_payoffs(lib)

74×74 Matrix{Int64}:
  0  -1  -1  -1  -1  -1  -1   1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   0  -1  -1  -1  -1  -1  -1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   0  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   0  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   0  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   0  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   1   0  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   1   1   1   1   1   1   0  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1   1   1   1   1   1   1   0     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1  -1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1   1   1  -1   1   1   1   1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1  -1   1   1  -1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1   1  -1  -1   1   1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  ⋮ 

In [13]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

spud2int (generic function with 1 method)

In [14]:
s2ind = Dict(lib[i] => i for i in 1:length(lib));
int2ind = Dict(spud2int(lib[i]) => i for i in 1:length(lib));

In [15]:
libi = [spud2int(ff) for ff in lib];

In [16]:
int2ind[313111]

73

In [17]:
as = [1,2,3]
as[end]

3

In [18]:
function eval_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    if length(bs)==0 || length(as)==0
        return 0
    end
    a = as[end]
    flag = true
    count = 0
    while flag
        if payoffs[a, bs[end-count]]==1
            count += 1
        else
            flag = false
        end
        if count == length(bs)
            flag = false
        end
    end
    return count
end

eval_beatn (generic function with 1 method)

In [19]:
as = [int2ind[ff] for ff in [214111, 112222, 313111, 113311]]
bs = [int2ind[ff] for ff in [121411, 121321, 113212, 213121]]

4-element Vector{Int64}:
 41
 40
 27
 65

In [20]:
eval_beatn(payoffs, as[1:3], bs[1:2])

2

In [21]:
eval_beatn(payoffs, bs[1:3], as[1:3])

2

In [22]:
eval_beatn(payoffs, as[1:4], bs[1:3])

2

In [23]:
eval_beatn(payoffs, bs[1:4], as[1:4])

3

In [24]:
size(payoffs)[1]

74

In [25]:
# tries all possible moves as extension of as
function try_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    ans = Array{Int64}(undef, size(payoffs)[1])
    for i in 1:size(payoffs)[1]
        as2 = copy(as)
        append!(as2, i)
        ans[i] = eval_beatn(payoffs, as2, bs)
    end
    return ans
end

try_beatn (generic function with 1 method)

In [26]:
for i in try_beatn(payoffs, as, bs)
    print(i)
end

00000000000000000000000011120110021211220011100001110000000000000120000000

In [27]:
# returns 1 if Player A lost
function lose_or_not(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    if maximum(scores) < prev_score
        return 1
    end
    return 0
end

lose_or_not (generic function with 1 method)

In [28]:
lose_or_not(payoffs, as[1:2], bs[1:2])

0

In [29]:
lose_or_not(payoffs, as[1:3], bs[1:3])

0

In [30]:
lose_or_not(payoffs, as[1:4], bs[1:4])

1

In [31]:
function legal_moves(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return cands
end

legal_moves (generic function with 1 method)

In [32]:
# picks randomly
function choose_random(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return rand(cands)
end

choose_random (generic function with 1 method)

In [33]:
# picks aggressively
function choose_aggro(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .== maximum(scores))
    return rand(cands)
end

choose_aggro (generic function with 1 method)

In [34]:
function play_random_vs_aggro(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_aggro(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function play_random_vs_random(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_random(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function display_game(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})
    for i in 1:length(as)
        print(libi[as[i]])
        print(" {")
        n = eval_beatn(payoffs, as[1:i], bs[1:(i-1)])
        print(n)
        print("}. ")
        if i <= length(bs)
            print(libi[bs[i]])
            print(" {")
            n = eval_beatn(payoffs, bs[1:i], as[1:i])
            print(n)
            print("}. ")
        end
        println()
    end
end

display_game (generic function with 1 method)

In [35]:
a_win, as, bs = play_random_vs_aggro(payoffs, [int2ind[111133]], [int2ind[222211]])
display_game(payoffs, as, bs)

111133 {0}. 222211 {1}. 
213112 {1}. 213211 {2}. 
113221 {2}. 214111 {3}. 
123211 {3}. 113311 {3}. 
122212 {4}. 121312 {4}. 
121411 {5}. 121123 {5}. 


In [36]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# for a in 1:nlib
#     for b in 1:nlib
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a], [b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [37]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)
#     js = findall(wins[i, :].==minimum(wins[i, :]))
#     print(libi[i])
#     print(" ")
#     print(libi[js])
#     print(" ")
#     println(minimum(wins[i, :])/n_games)
# end

In [38]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a_moves = legal_moves(payoffs, [a1], [b1])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1], [a1, a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a], [b1,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [39]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [40]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2],[b1,b2])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
111322 {0}. 213211 {2}. 
131212 {3}. 121123 {3}. 
131122 {4}. 


In [41]:
# n_games = 1000
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a2 = int2ind[213112]
# b2 = int2ind[313111]
# a_moves = legal_moves(payoffs, [a1,a2], [b1,b2])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1,b2], [a1,a2,a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a], [b1,b2,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [42]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [43]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3],[b1,b2,b3])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
111322 {1}. 111421 {2}. 
112213 {2}. 213112 {3}. 
122212 {3}. 112321 {4}. 
113212 {5}. 112312 {5}. 


In [44]:
# n_games = 100
# counts = Array{Int64}(undef, (nlib, nlib)) .* 0
# wins = Array{Int64}(undef, (nlib, nlib)) .* 0
# a1 = int2ind[121321]
# b1 = int2ind[214111]
# a2 = int2ind[213112]
# b2 = int2ind[313111]
# a3 = int2ind[121321]
# b3 = int2ind[121222]
# a_moves = legal_moves(payoffs, [a1,a2,a3], [b1,b2,b3])
# for a in a_moves
#     b_moves = legal_moves(payoffs, [b1,b2,b3], [a1,a2,a3,a])
#     for b in b_moves
#         for i in 1:n_games
#             a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a], [b1,b2,b3,b])
#             counts[a,b] += 1
#             if a_win==1
#                 wins[a,b] +=1
#             end
#         end
        
#     end
# end

In [45]:
# maxes = [minimum(wins[i, :]) for i in 1:nlib]
# for i in sortperm(-maxes)[1:10]
#     if i in a_moves
#         wns = wins[i, :]
#         cns = counts[i, :]
#         min_wins = minimum(wns[cns .> 0])
#         js = findall(wns .== min_wins)
#         print(libi[i])
#         print(" ")
#         print(libi[js])
#         print(" ")
#         println(minimum(wins[i, :])/n_games)
#     end
# end

In [46]:
# check game lengths
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a4 = int2ind[113122]
b4 = int2ind[111133]
#a_win, as, bs = play_random_vs_aggro(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
a_win, as, bs = play_random_vs_random(payoffs, [a1,a2,a3,a4],[b1,b2,b3,b4])
display_game(payoffs, as, bs)

121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 111133 {2}. 
111232 {2}. 122212 {2}. 
212122 {2}. 214111 {3}. 


## MCTS

In [47]:
function intarr_to_string(as::Array{Int64})::String
    return join([convert(Char, a+32) for a in as])
end

function string_to_intarr(s::String)::Array{Int64}
    return [convert(Int64, s[i])-32 for i in 1:length(s)]
end

# s=intarr_to_string([i for i in 1:75])
# as=string_to_intarr(s)
# for i in as
#     print(i)
#     print(" ")
# end

function split_arrs(zs::Array{Int64})::Tuple{Array{Int64},Array{Int64}}
    if mod(length(zs), 2)==0
        return (zs[1:2:(end-1)], zs[2:2:end])
    else
        return (zs[1:2:end], zs[2:2:(end-1)])
    end
end

function merge_arrs(as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    zs = Array{Int64}(undef, length(as)+length(bs))
    if mod(length(zs), 2)==0
        zs[1:2:(end-1)]=as
        zs[2:2:end]=bs
    else
        zs[1:2:end] = as
    zs[2:2:(end-1)] = bs
    end
    return zs
end

merge_arrs (generic function with 1 method)

In [48]:
function propagate_win(tree::Dict{String, Vector{Int64}}, leaf::String, win::Int64)::Dict{String, Vector{Int64}}
    for i in 0:length(leaf)
        if i==0
            node=""
        else
            node=leaf[1:i]
        end
        if haskey(tree, node)
            tree[node][2] += 1
            tree[node][1] += win
        end
    end
    return tree
end

function append_move_to_leaf(leaf::String, move::Int64)::String
    zs = string_to_intarr(leaf)
    append!(zs, move)
    return intarr_to_string(zs)
end

function get_children(payoffs::Array{Int64}, leaf::String)::Array{String}
    # get arrays
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate successors
    if player==1
        cands = legal_moves(payoffs, as, bs)
    else
        cands = legal_moves(payoffs, bs, as)
    end
    return [append_move_to_leaf(leaf, c) for c in cands]
end

get_children (generic function with 1 method)

In [49]:
function check_and_propagate_terminal(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    children = get_children(payoffs, leaf)
    if length(children) == 0
        # indicate terminal
        tree[leaf][3]=1
        if mod(length(leaf), 2)==0
            #p1 lost
            tree = propagate_win(tree, leaf, 0)
        else
            #p1 won
            tree = propagate_win(tree, leaf, 1)
        end
    end
    return tree
end

check_and_propagate_terminal (generic function with 1 method)

In [50]:
function try_add_node(tree::Dict{String, Vector{Int64}}, node::String)::Dict{String, Vector{Int64}}
    if haskey(tree, node)
        return tree
    else
        tree[node] = [0,0,0]
        return tree
    end
end

function query_node(tree, node)::Array{Int64}
    if haskey(tree, node)
        return tree[node]
    else
        return [0,0,0]
    end
end

query_node (generic function with 1 method)

In [51]:
function expand_leaf(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)::Dict{String, Vector{Int64}}
    # check if leaf is terminal
    tree = check_and_propagate_terminal(payoffs, tree, leaf)
    if tree[leaf][3]==1
        return tree
    end
    # get arrays for leaf
    as, bs = split_arrs(string_to_intarr(leaf))
    if mod(length(leaf), 2)==0
        player = 1
    else
        player = 2
    end
    # generate random successor
    if player==1
        cands = legal_moves(payoffs, as, bs)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    else
        cands = legal_moves(payoffs, bs, as)
        newleaf = append_move_to_leaf(leaf, rand(cands))
    end
    tree = try_add_node(tree, newleaf)
    tree = check_and_propagate_terminal(payoffs, tree, newleaf)
    # play the match
    as, bs = split_arrs(string_to_intarr(newleaf))
    a_win, as2, bs2 = play_random_vs_random(payoffs, as, bs)
    # propagate result
    if a_win == 1
        tree = propagate_win(tree, newleaf, 1)
    else
        tree = propagate_win(tree, newleaf, 0)
    end
    return tree
end


expand_leaf (generic function with 1 method)

In [52]:
function sample_next_node(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String, 
        explore_const::Float64=1.4)::String
    # values for UCT
    n = tree[node][2]
    # assumes that all children are added
    children = get_children(payoffs, node)
    counts = [query_node(tree, c)[2] for c in children]
    wins = [query_node(tree, c)[1] for c in children]
    if minimum(counts) == 0
        sel = children[rand(findall(counts .== 0))]
        return sel
    end
    if mod(length(node), 2)==1
        wins = counts .- wins
    end
    scores = counts .* 0.0
    for i in 1:length(scores)
        if counts[i] > 0
            scores[i] = wins[i]/counts[i] + explore_const * sqrt((log(n)/counts[i]))
        end
    end
    sel = children[rand(findall(scores .== maximum(scores)))]
    return sel
end

sample_next_node (generic function with 2 methods)

In [53]:
function random_leaf(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String="")::String
    while query_node(tree,node)[2] > 1 && query_node(tree,node)[3] != 1
        node = sample_next_node(payoffs, tree, node)
    end
    return node
end

random_leaf (generic function with 2 methods)

In [54]:
# exploit mode
function sample_next_node2(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String, beta::Float64 = 1.0)::String
    # values for UCT
    n = tree[node][2]
    # assumes that all children are added
    children = get_children(payoffs, node)
    counts = [query_node(tree, c)[2] for c in children]
    wins = [query_node(tree, c)[1] for c in children]
    if mod(length(node), 2)==1
        wins = counts .- wins
    end
    scores = counts .* 0.0
    for i in 1:length(scores)
        if counts[i] > 0
            scores[i] = wins[i]/counts[i]
        end
    end
    scores = [exp(beta*x) for x in scores]
    scores = scores./sum(scores)
    cs = cumsum(scores)
    z = rand(1)[1]
    sel = children[minimum(findall(cs .>= z))]
    return sel
end

sample_next_node2 (generic function with 2 methods)

In [55]:
function display_node(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    print("Node: ")
    println(node)
    as, bs = split_arrs(string_to_intarr(node))
    display_game(payoffs, as, bs)
    print("Stats: ")
    print(query_node(tree,node)[1])
    print("/")
    print(query_node(tree,node)[2])
    print("=")
    s = string(query_node(tree,node)[1]/query_node(tree,node)[2])
    println(s[1:min(5, length(s))])
    if query_node(tree,node)[3]==1
        println("[terminal] ")
    end
end

function display_path(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, leaf::String)
    for i in 0:length(leaf)
        if i==0
            if haskey(tree, "")
                display_node(payoffs, tree, "")
                println()
            end
        else
            if haskey(tree, leaf[1:i])
                display_node(payoffs, tree, leaf[1:i])
                println()
            end
        end
    end
end

function display_choices(payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String)
    children = get_children(payoffs, node)
    display_node(payoffs, tree, node)
    for c in children
        println()
        display_node(payoffs, tree, c)
    end
end

display_choices (generic function with 1 method)

In [56]:
a1 = int2ind[121321]
b1 = int2ind[214111]
a2 = int2ind[213112]
b2 = int2ind[313111]
a3 = int2ind[121321]
b3 = int2ind[121222]
a4 = int2ind[113122]
b4 = int2ind[111133]
s0=intarr_to_string(merge_arrs([a1,a2,a3,a4],[b1,b2,b3,b4]))

"Hc`iHE9\""

In [57]:
function create_tree(node::String = "")::Dict{String, Vector{Int64}}
    return Dict(node => [0,0,0])
end

create_tree (generic function with 2 methods)

In [58]:
#tree = load_object("beatn10_tree.jld2")

tree = create_tree()


Dict{String, Vector{Int64}} with 1 entry:
  "" => [0, 0, 0]

In [59]:
#tree = expand_leaf(payoffs, tree, "")

In [60]:
node = random_leaf(payoffs, tree, "")

""

In [61]:
function grow_tree(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}, node::String = "")::Dict{String, Vector{Int64}}
    node = random_leaf(payoffs, tree, node)
    tree = try_add_node(tree, node)
    tree = check_and_propagate_terminal(payoffs, tree, node)
    tree = expand_leaf(payoffs, tree, node)
    return tree
end

function grow_tree(
        payoffs::Array{Int64}, tree::Dict{String, Vector{Int64}}; 
        node::String = "", iters::Int64 = 1000)::Dict{String, Vector{Int64}}
    for i in 1:iters
        tree = grow_tree(payoffs, tree, node)
    end
    return tree
end

grow_tree (generic function with 2 methods)

In [62]:
@time tree = grow_tree(payoffs, tree, iters=100)

  0.057511 seconds (617.82 k allocations: 48.703 MiB, 10.93% gc time, 64.74% compilation time)


Dict{String, Vector{Int64}} with 172 entries:
  "Z"   => [1, 1, 0]
  "1"   => [0, 1, 0]
  "V3"  => [1, 1, 0]
  "+j"  => [1, 1, 0]
  "OM"  => [1, 1, 0]
  "="   => [1, 2, 0]
  "'"   => [0, 1, 0]
  "a;"  => [0, 1, 0]
  "*"   => [0, 1, 0]
  "bf"  => [1, 1, 0]
  ")"   => [0, 1, 0]
  "K"   => [1, 2, 0]
  ":@"  => [1, 1, 0]
  ")%"  => [0, 1, 0]
  "`E"  => [1, 1, 0]
  "9"   => [0, 1, 0]
  "F"   => [1, 2, 0]
  "c"   => [0, 1, 0]
  "!]"  => [1, 1, 0]
  "',"  => [0, 1, 0]
  "\$G" => [0, 1, 0]
  "["   => [1, 2, 0]
  "Qf"  => [1, 1, 0]
  "g"   => [1, 2, 0]
  "hd"  => [1, 1, 0]
  ⋮     => ⋮

In [63]:
# for metaiter in 1:10
#     s = ""
#     for iter in 1:100000
#         s = string(s,".")
#         node = random_leaf(payoffs, tree, "")
#         tree = try_add_node(tree, node)
#         tree = check_and_propagate_terminal(payoffs, tree, node)
#         s = string(s, node, ".")
#         tree = expand_leaf(payoffs, tree, node)
#     end
#     println(s[end-100:end])
#     save_object("beatn10_tree.jld2", tree)
# end

In [64]:
length(tree)

172

In [65]:
node = random_leaf(payoffs, tree, "")

"N"

In [66]:
# for mj in 1:2
#     nodes = Array{String}(undef, 50)
#     for i in 1:length(nodes)
#         node = ""
#         for j in 1:mj
#             if haskey(tree, node)
#                 node = sample_next_node2(payoffs, tree, node, 10.0)
#             end
#         end
#         nodes[i] = node
#     end
#     nodes = sort(unique(nodes))
#     for node in nodes
#         display_node(payoffs, tree, node)
#         println()
#     end
# end

In [67]:
# node = random_leaf(payoffs, tree, "")
# display_path(payoffs, tree, node)

In [68]:
# node = ""
# while query_node(tree,node)[2] > 1 && query_node(tree,node)[3] != 1
#     node = sample_next_node(payoffs, tree, node)
#     println(node)
# end

In [69]:
# display_node(payoffs, tree, node)

In [70]:
# children = get_children(payoffs, node)
# children

In [71]:
# sample_next_node(payoffs, tree, node)

In [72]:
#display_choices(payoffs, tree, "HG`i\"")

### Tree at arbitrary root

In [73]:
length(s0)

8

In [80]:
node0 = s0[1:7]
tree = create_tree(node0)
@time tree = grow_tree(payoffs, tree, node=node0, iters=10000)

  2.105283 seconds (45.69 M allocations: 4.853 GiB, 18.60% gc time, 0.41% compilation time)


Dict{String, Vector{Int64}} with 17059 entries:
  "Hc`iHE9Cf2"    => [1, 1, 0]
  "Hc`iHE9<+"     => [1, 2, 0]
  "Hc`iHE92WX"    => [1, 1, 0]
  "Hc`iHE9V`"     => [1, 1, 0]
  "Hc`iHE9Y>U"    => [1, 1, 0]
  "Hc`iHE9..9"    => [1, 1, 0]
  "Hc`iHE9+gc"    => [0, 1, 0]
  "Hc`iHE913"     => [1, 1, 0]
  "Hc`iHE9\"FGi;" => [0, 1, 0]
  "Hc`iHE9\"h>D"  => [1, 1, 0]
  "Hc`iHE9ib<c"   => [0, 1, 0]
  "Hc`iHE96QS;"   => [0, 1, 0]
  "Hc`iHE9&fcL"   => [1, 1, 0]
  "Hc`iHE96L&"    => [1, 1, 0]
  "Hc`iHE93cDG"   => [1, 1, 0]
  "Hc`iHE9\"4>J"  => [0, 1, 0]
  "Hc`iHE9`e8"    => [1, 1, 0]
  "Hc`iHE9RcY"    => [1, 1, 0]
  "Hc`iHE9bd["    => [1, 1, 0]
  "Hc`iHE9\"hcC"  => [1, 2, 0]
  "Hc`iHE9W<"     => [1, 1, 0]
  "Hc`iHE9\"iT^"  => [1, 3, 0]
  "Hc`iHE965d"    => [1, 1, 0]
  "Hc`iHE9;J3("   => [1, 1, 0]
  "Hc`iHE9[GY"    => [1, 1, 0]
  ⋮               => ⋮

In [81]:
for mj in 1:2
    nodes = Array{String}(undef, 50)
    for i in 1:length(nodes)
        node = node0
        for j in 1:mj
            if haskey(tree, node)
                node = sample_next_node2(payoffs, tree, node, 10.0)
            end
        end
        nodes[i] = node
    end
    nodes = sort(unique(nodes))
    for node in nodes
        display_node(payoffs, tree, node)
        println()
    end
end

Node: Hc`iHE9"
121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 111133 {2}. 
Stats: 409/1301=0.314

Node: Hc`iHE9*
121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 111331 {0}. 
Stats: 79/143=0.552

Node: Hc`iHE9+
121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 111412 {0}. 
Stats: 150/354=0.423

Node: Hc`iHE94
121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 112231 {0}. 
Stats: 57/87=0.655

Node: Hc`iHE95
121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 112312 {0}. 
Stats: 114/242=0.471

Node: Hc`iHE99
121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 113122 {0}. 
Stats: 76/135=0.562

Node: Hc`iHE9;
121321 {0}. 214111 {0}. 
213112 {0}. 313111 {0}. 
121321 {0}. 121222 {0}. 
113122 {0}. 113212 {1}. 
Stats: 82/150=0.546

Node: Hc`iHE9@
121321 {0}. 214111 {0}. 
21311

## Play vs MCTS
### CPU first

In [82]:
n_mc = 100000
beta = 20.0
game = ""
tree = create_tree(game)
display_node(payoffs, create_tree(game), game)

Node: 
Stats: 0/0=NaN


In [84]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, node=game, iters=n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, create_tree(game), game)

 20.490611 seconds (558.40 M allocations: 42.530 GiB, 26.56% gc time)
Node: @
114211 {0}. 
Stats: 0/0=NaN


In [85]:
# player turn
pmove = int2ind[313111]
game = append_move_to_leaf(game, pmove)
display_node(payoffs, create_tree(game), game)

Node: @i
114211 {0}. 313111 {0}. 
Stats: 0/0=NaN


In [87]:
node = game[1:2]
display_node(payoffs, create_tree(node), node)

Node: @i
114211 {0}. 313111 {0}. 
Stats: 0/0=NaN


In [88]:
tree = create_tree(node)
@time tree = grow_tree(payoffs, tree, node=node, iters=n_mc)

 19.393523 seconds (441.44 M allocations: 34.339 GiB, 26.73% gc time)


Dict{String, Vector{Int64}} with 177624 entries:
  "@i/M@H"    => [0, 1, 0]
  "@i/%`<"    => [0, 1, 0]
  "@i*=S^"    => [0, 1, 0]
  "@i*,RK"    => [1, 1, 0]
  "@ihNaR"    => [0, 1, 0]
  "@i/*]"     => [0, 1, 0]
  "@i55]"     => [1, 1, 0]
  "@i)(eG"    => [0, 1, 0]
  "@i*SJ"     => [0, 1, 0]
  "@i\"'>"    => [0, 1, 0]
  "@iIJ/"     => [0, 1, 0]
  "@i\\%fh"   => [1, 1, 0]
  "@i\"@^"    => [1, 1, 0]
  "@i*RF"     => [0, 1, 0]
  "@i*?Zi"    => [0, 1, 0]
  "@i0D<"     => [0, 2, 0]
  "@i+(a"     => [1, 1, 0]
  "@i[E;DT<(" => [0, 1, 0]
  "@i\"O`["   => [1, 1, 0]
  "@i)FAY"    => [1, 1, 0]
  "@ii*VU"    => [1, 1, 0]
  "@ifS<@3"   => [0, 1, 0]
  "@ii6]\$"   => [1, 1, 0]
  "@i8aTM"    => [0, 1, 0]
  "@ijG?!"    => [0, 1, 0]
  ⋮           => ⋮

### MCTS vs self

In [356]:
n_mc = 100000
beta = 20.0
game = ""
tree = create_tree(game)
display_node(payoffs, tree, game)

Node: 
Stats: 0/0=NaN


In [357]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 56.583590 seconds (558.71 M allocations: 42.540 GiB, 34.94% gc time)
Node: `
213112 {0}. 
Stats: 813/1705=0.476


In [358]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 54.774908 seconds (495.92 M allocations: 38.330 GiB, 37.49% gc time)
Node: `i
213112 {0}. 313111 {0}. 
Stats: 4529/11739=0.385


In [359]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 58.743900 seconds (510.55 M allocations: 40.618 GiB, 36.95% gc time)
Node: `ih
213112 {0}. 313111 {0}. 
312211 {0}. 
Stats: 241/677=0.355


In [360]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 26.903677 seconds (234.21 M allocations: 18.199 GiB, 36.70% gc time)
Node: `ihU
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
Stats: 3833/142574=0.026


In [361]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 14.737715 seconds (129.20 M allocations: 10.370 GiB, 34.92% gc time)
Node: `ihUX
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 
Stats: 1800/63040=0.028


In [362]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 13.172859 seconds (111.80 M allocations: 8.985 GiB, 35.84% gc time)
Node: `ihUX<
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
Stats: 188/4048=0.046


In [363]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 14.017991 seconds (121.78 M allocations: 10.367 GiB, 32.63% gc time)
Node: `ihUX<R
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
131122 {3}. 
Stats: 1349/103227=0.013


In [364]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 13.121446 seconds (111.25 M allocations: 9.680 GiB, 33.92% gc time)
Node: `ihUX<Rd
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
131122 {3}. 222112 {3}. 
Stats: 150/2233=0.067


In [365]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 16.123942 seconds (132.70 M allocations: 12.364 GiB, 32.35% gc time)
Node: `ihUX<Rd\
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
131122 {3}. 222112 {3}. 
212131 {3}. 
Stats: 294/27586=0.010


In [367]:
# MCTS turn
tree = create_tree(game)
@time tree = grow_tree(payoffs, tree, game, n_mc)
game = sample_next_node2(payoffs, tree, game, beta)
display_node(payoffs, tree, game)

 10.299981 seconds (86.84 M allocations: 7.930 GiB, 34.29% gc time)
Node: `ihUX<Rd\>
213112 {0}. 313111 {0}. 
312211 {0}. 131311 {2}. 
141112 {2}. 113221 {3}. 
131122 {3}. 222112 {3}. 
212131 {3}. 114112 {5}. 
Stats: 0/12978=0.0
[terminal] 


In [366]:
node = "<Bi;HR\"`"
display_node(payoffs, create_tree(node), node)

Node: <Bi;HR"`
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
Stats: 0/0=NaN


In [355]:
tree = create_tree(node)
@time tree = grow_tree(payoffs, tree, node, n_mc)
display_choices(payoffs, tree, node)

  5.259248 seconds (51.59 M allocations: 4.324 GiB, 32.27% gc time)
Node: <Bi;HR"`
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
Stats: 199854/199905=0.999

Node: <Bi;HR"`D
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
121213 {4}. 
Stats: 99894/99894=1.0
[terminal] 

Node: <Bi;HR"`G
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
121312 {3}. 
Stats: 64/115=0.556

Node: <Bi;HR"`J
113221 {0}. 121123 {1}. 
313111 {1}. 113212 {1}. 
121321 {2}. 131122 {3}. 
111133 {3}. 213112 {3}. 
122113 {4}. 
Stats: 99896/99896=1.0
[terminal] 


In [341]:
display_choices(payoffs, tree, "b`HG>ii")

Node: b`HG>ii
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 
Stats: 5294/13927=0.380

Node: b`HG>ii!
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111124 {0}. 
Stats: 54/93=0.580

Node: b`HG>ii"
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111133 {0}. 
Stats: 116/308=0.376

Node: b`HG>ii#
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111142 {0}. 
Stats: 71/145=0.489

Node: b`HG>ii$
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111214 {0}. 
Stats: 46/72=0.638

Node: b`HG>ii%
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111223 {0}. 
Stats: 58/104=0.557

Node: b`HG>ii&
213211 {0}. 213112 {0}. 
121321 {0}. 121312 {0}. 
114112 {0}. 313111 {0}. 
313111 {0}. 111232 {0}. 
Stats: 90/209=0.430

Node: b`HG>ii'
213211 {0}. 213112 {0}. 
121321 {0}. 121312

## Mate in X analysis

In [211]:
function mate_checker(
        payoffs::Array{Int64},  
        node00::String;
        nits_tree::Int64 = 3,
        nits_cands::Int64 = 1000,
        verbose::Bool = true
        )::Tuple{Float64, Int64}
    tree = create_tree(node00)
    for i in 1:nits_tree
        tree = expand_game_tree(payoffs, tree)
    end
    # Check mate-in-X property
    lens = [length(k) for k in keys(tree)]
    minlen = minimum(lens)
    maxlen = maximum(lens)
    game_values = Dict(node00 => 0.0)
    for len in maxlen:-1:minlen
        wvalue = -1.0 # if len is even and node is terminal, then pl 1 loses
        if mod(len, 2)==1
            wvalue = 1.0
        end
        for node in [nd for nd in keys(tree) if length(nd)==len]
            if tree[node][3]==1
                game_values[node]=wvalue
            elseif length(node)==maxlen
                game_values[node]=0
            else
                children=get_children(payoffs, node)
                vals = [game_values[c] for c in children]
                if mod(len, 2)==1
                    # player 2 to move: get minimum of children values
                    game_values[node] = minimum(vals)
                else
                    # player 1 to move: get maximum of children values
                    game_values[node] = maximum(vals)
                end
            end
        end
    end
    # check length of best resistance
    best_res_length = Dict{String,Int64}()
    val = game_values[node00]
    if val > 0
        resister = 1
    else
        resister = 0 # player 2
    end
    for iter in 1:nits_cands
        node = node00
        val = game_values[node00]
        flag = true
        while flag
            cands = [c for c in keys(game_values) if length(c)==(length(node)+1) && c[1:length(node)]==node && game_values[c]==val]
            if length(cands) > 0
                node = rand(cands)
            else
                flag = false
            end
        end
        best_res_length[node] = length(node)-minlen
    end
    # Range of BRs
    lens_br = [length(k) for k in keys(best_res_length)]
    minlen_br = minlen
    maxlen_br = maximum(lens_br)
    for len in maxlen_br:-1:minlen_br
        nodes = unique([node[1:len] for node in keys(best_res_length) if length(node) >= len])
        for node in nodes
            if haskey(best_res_length, node)
                # do nothing
            else
                children = [c for c in keys(best_res_length) if length(c) == (length(node)+1) && c[1:length(node)]==node]
                brs = [best_res_length[c] for c in children]
                if mod(length(node), 2)==resister
                    # current node is resister to play
                    best_res_length[node] = maximum(brs)
                else
                    best_res_length[node] = minimum(brs)                
                end
            end
        end
    end
    if verbose
        node = node00
        br = best_res_length[node00]
        flag = true
        while flag
            cands = [nd for nd in keys(best_res_length) if length(nd)==(length(node)+1) && best_res_length[nd]==br && nd[1:length(node)]==node]
            if length(cands)==0
                flag = false
            else
                node = rand(cands)
            end
        end
        display_path(payoffs, tree, node)
    end
    return (game_values[node00], best_res_length[node00])
end

mate_checker (generic function with 1 method)

In [212]:
function expand_game_tree(payoffs::Array{Int64},  tree::Dict{String, Vector{Int64}})::Dict{String, Vector{Int64}}
    maxlen = maximum([length(nd) for nd in keys(tree)])
    nodes = [nd for nd in keys(tree) if length(nd)==maxlen]
    for node in nodes
        children = get_children(payoffs, node)
        for c in children
            tree = try_add_node(tree, c)
            tree = check_and_propagate_terminal(payoffs, tree, c)
        end
    end
    return tree
end

expand_game_tree (generic function with 1 method)

In [213]:
# a game where P1 lost
node0 = "%ZU7C`:#"
display_node(payoffs, create_tree(node0), node0)

Node: %ZU7C`:#
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
Stats: 0/0=NaN


In [214]:
# find where a mate in X occurs
node00 = node0[1:7]
display_node(payoffs, create_tree(node00), node00)

Node: %ZU7C`:
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 
Stats: 0/0=NaN


In [215]:
@time mate_checker(payoffs, node00, nits_tree=4, verbose=true)

Node: %ZU7C`:
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 
Stats: 158/201=0.786

Node: %ZU7C`:+
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111412 {3}. 
Stats: 0/12=0.0

Node: %ZU7C`:+B
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111412 {3}. 
121123 {3}. 
Stats: 0/2=0.0

Node: %ZU7C`:+BE
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111412 {3}. 
121123 {3}. 121222 {5}. 
Stats: 0/1=0.0
[terminal] 

  0.700353 seconds (7.32 M allocations: 255.038 MiB, 5.07% gc time, 44.94% compilation time)


(-1.0, 3)

#### debugging code

In [167]:
node00 = "%ZU7C`:#"
tree = create_tree(node00)
for i in 1:3
    tree = expand_game_tree(payoffs, tree)
end
display_node(payoffs, tree, node00)

Node: %ZU7C`:#
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
Stats: 81/82=0.987


In [174]:
[nd for nd in keys(tree) if tree[nd][3]==1 && length(nd)==11]

81-element Vector{String}:
 "%ZU7C`:#E2c"
 "%ZU7C`:#E&a"
 "%ZU7C`:#;+c"
 "%ZU7C`:#E38"
 "%ZU7C`:#E3c"
 "%ZU7C`:#E&8"
 "%ZU7C`:#L+a"
 "%ZU7C`:#;5c"
 "%ZU7C`:#G+c"
 "%ZU7C`:#aE;"
 "%ZU7C`:#E5a"
 "%ZU7C`:#L)c"
 "%ZU7C`:#;%c"
 ⋮
 "%ZU7C`:#E;c"
 "%ZU7C`:#G+a"
 "%ZU7C`:#;)a"
 "%ZU7C`:#G5b"
 "%ZU7C`:#G)c"
 "%ZU7C`:#G%a"
 "%ZU7C`:#E+b"
 "%ZU7C`:#E5b"
 "%ZU7C`:#L3c"
 "%ZU7C`:#E58"
 "%ZU7C`:#E]c"
 "%ZU7C`:#G)b"

In [175]:
display_path(payoffs, tree, "%ZU7C`:#G5b")

Node: %ZU7C`:#
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
Stats: 81/82=0.987

Node: %ZU7C`:#G
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
121312 {3}. 
Stats: 12/12=1.0

Node: %ZU7C`:#G5
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
121312 {3}. 112312 {4}. 
Stats: 3/3=1.0

Node: %ZU7C`:#G5b
111223 {0}. 212113 {1}. 
131311 {1}. 112411 {1}. 
121132 {1}. 213112 {1}. 
113131 {3}. 111142 {3}. 
121312 {3}. 112312 {4}. 
213211 {5}. 
Stats: 1/1=1.0
[terminal] 



In [194]:
# Check mate-in-X property
lens = [length(k) for k in keys(tree)]
minlen = minimum(lens)
maxlen = maximum(lens)
game_values = Dict(node00 => 0.0)
for len in maxlen:-1:minlen
    wvalue = -1.0 # if len is even and node is terminal, then pl 1 loses
    if mod(len, 2)==1
        wvalue = 1.0
    end
    for node in [nd for nd in keys(tree) if length(nd)==len]
        if tree[node][3]==1
            game_values[node]=wvalue
        elseif length(node)==maxlen
            game_values[node]=0
        else
            children=get_children(payoffs, node)
            vals = [game_values[c] for c in children]
            if mod(len, 2)==1
                # player 2 to move: get minimum of children values
                game_values[node] = minimum(vals)
            else
                # player 1 to move: get maximum of children values
                game_values[node] = maximum(vals)
            end
        end
    end
end

In [195]:
game_values["%ZU7C`:#G5b"]

1.0

In [196]:
game_values["%ZU7C`:#G5"]

1.0

In [197]:
[nd => game_values[nd] for nd in keys(game_values) if length(nd)==10 && nd[1:9]=="%ZU7C`:#G"]

4-element Vector{Pair{String, Float64}}:
 "%ZU7C`:#G%" => 1.0
 "%ZU7C`:#G+" => 1.0
 "%ZU7C`:#G5" => 1.0
 "%ZU7C`:#G)" => 1.0

In [198]:
game_values["%ZU7C`:#G"]

1.0

In [199]:
game_values[node00]

1.0

In [201]:
# check length of best resistance
best_res_length = Dict{String,Int64}()
val = game_values[node00]
if val > 0
    resister = 1
else
    resister = 0 # player 2
end

1

In [202]:
for iter in 1:100
    node = node00
    val = game_values[node00]
    flag = true
    while flag
        cands = [c for c in keys(game_values) if length(c)==(length(node)+1) && c[1:length(node)]==node && game_values[c]==val]
        if length(cands) > 0
            node = rand(cands)
        else
            flag = false
        end
    end
    best_res_length[node] = length(node)-minlen
end
best_res_length

Dict{String, Int64} with 38 entries:
  "%ZU7C`:#;5b" => 3
  "%ZU7C`:#L+c" => 3
  "%ZU7C`:#;%c" => 3
  "%ZU7C`:#aEL" => 3
  "%ZU7C`:#L%a" => 3
  "%ZU7C`:#L%b" => 3
  "%ZU7C`:#L+b" => 3
  "%ZU7C`:#;+b" => 3
  "%ZU7C`:#;+c" => 3
  "%ZU7C`:#;%a" => 3
  "%ZU7C`:#;)c" => 3
  "%ZU7C`:#L)a" => 3
  "%ZU7C`:#;)a" => 3
  "%ZU7C`:#;5a" => 3
  "%ZU7C`:#bE;" => 3
  "%ZU7C`:#G)c" => 3
  "%ZU7C`:#L+a" => 3
  "%ZU7C`:#G%b" => 3
  "%ZU7C`:#G%a" => 3
  "%ZU7C`:#L3b" => 3
  "%ZU7C`:#G5b" => 3
  "%ZU7C`:#L5c" => 3
  "%ZU7C`:#;5c" => 3
  "%ZU7C`:#aE;" => 3
  "%ZU7C`:#;)b" => 3
  ⋮             => ⋮

In [203]:
# Range of BRs
lens_br = [length(k) for k in keys(best_res_length)]
minlen_br = minlen
maxlen_br = maximum(lens_br)

11

In [204]:
for len in maxlen_br:-1:minlen_br
    nodes = unique([node[1:len] for node in keys(best_res_length) if length(node) >= len])
    for node in nodes
        if haskey(best_res_length, node)
            # do nothing
        else
            children = [c for c in keys(best_res_length) if length(c) == (length(node)+1) && c[1:length(node)]==node]
            brs = [best_res_length[c] for c in children]
            if mod(length(node), 2)==resister
                # current node is resister to play
                best_res_length[node] = maximum(brs)
            else
                best_res_length[node] = minimum(brs)                
            end
        end
    end
end

In [205]:
# node = node00
# val = game_values[node00]
# flag = true
# while flag
#     cands = [c for c in nodes_by_length[length(node)-minlen+2] if c[1:length(node)]==node && game_values[c]==val]
#     if length(cands) > 0
#         node = rand(cands)
#     else
#         flag = false
#     end
# end
# display_path(payoffs, tree, node)

In [206]:
[nd => best_res_length[nd] for nd in keys(best_res_length) if length(nd)==1]

Any[]

In [208]:
best_res_length[node00]

3

In [207]:
game_values[node00]

1.0

### automate finding mate-in-2 problems

In [221]:
n_mc = 1000
beta = 20.0
for iter in 1:20
    game = ""
    #tree = create_tree(game)
    flag = true
    while flag
        ## MCTS turn
        tree = create_tree(game)
        tree = grow_tree(payoffs, tree, node=game, iters=n_mc)
        game = sample_next_node2(payoffs, tree, game, beta)
#         children = get_children(payoffs, game)
#         cands = Array{String}(undef, length(children))
#         nc = 0
#         # eliminate mates in 1, if possible
#         for c in children
#             value, br = mate_checker(payoffs, string(c), nits_tree = 1, verbose=false)
#             if (value == -1.0 && mod(length(c), 2)==1) || (value == 1.0 && mod(length(c), 2)==0)
#             else
#                 nc += 1
#                 cands[nc] = c
#             end
#         end
#         cands = cands[1:nc]
#         if length(cands) > 0
#             game = rand(cands)
#         else
#             println("Unavoidable mate")
#             display_node(payoffs, create_tree(game), game)
#             game = rand(children)
#         end
        if length(get_children(payoffs, game)) < 20
            print("Checking mate... ")
            #display_node(payoffs, tree, game)
            @time value, br = mate_checker(payoffs, game, nits_tree = 3, verbose=false)
            if abs(value)==1.0 && br == 3
                flag = false
                println("----MATE-IN-2 FOUND---")
                value, br = mate_checker(payoffs, game, nits_tree = 1, verbose=true)
                #print("value:")
                #print(value)
                #print(" best resist:")
                #println(br)
            end
        end
        if length(get_children(payoffs, game))==0
            flag = false
        end
    end
end

Checking mate...   0.002385 seconds (70.40 k allocations: 2.272 MiB)
Checking mate...   0.001754 seconds (52.91 k allocations: 1.713 MiB)
Checking mate...   0.000163 seconds (3.53 k allocations: 135.758 KiB)
Checking mate...   0.027229 seconds (492.29 k allocations: 16.621 MiB, 23.00% gc time)
Checking mate...   0.017257 seconds (385.40 k allocations: 13.145 MiB)
Checking mate...   0.000178 seconds (3.53 k allocations: 139.328 KiB)
Checking mate...   0.001601 seconds (43.23 k allocations: 1.541 MiB)
Checking mate...   0.004243 seconds (100.44 k allocations: 3.435 MiB)
Checking mate...   0.000168 seconds (3.53 k allocations: 139.328 KiB)
Checking mate...   0.000182 seconds (3.53 k allocations: 139.375 KiB)
Checking mate...   0.078399 seconds (1.67 M allocations: 52.904 MiB, 7.54% gc time)
Checking mate...   0.000182 seconds (3.53 k allocations: 135.711 KiB)
Checking mate...   0.102658 seconds (2.13 M allocations: 72.030 MiB, 6.05% gc time)
Checking mate...   0.118827 seconds (1.93 M all